In [ ]:
!pip install python-igraph
!pip install cairocffi
!pip install https://github.com/matplotlib/basemap/archive/master.zip

In [ ]:
import igraph as ig
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cairocffi
import requests
import json
from itertools import islice
from mpl_toolkits import basemap
import networkx as nx

## Importing, Inspecting and Visualizing the Facebook Network

In [ ]:
el = pd.read_csv('https://raw.githubusercontent.com/swapnilsayansaha/Social_Complex_Nets/main/Project2/facebook_combined.txt',
                 sep=' ',header=None) #import the list of edges
g = ig.Graph.DataFrame(el, directed=False) #generate undirected network from the edge list
el.head() #see how the data looks like

In [ ]:
print("Number of nodes:",g.vcount()) #Print number of vertices
print("Number of edges:", g.ecount()) #Print number of edges
print("Is the graph connected?",g.is_connected()) #Check if the entire graph is connected
print("Average degree: ", np.mean(g.degree())) #Average degree of a node

In [ ]:
x = g.degree_distribution() #get the degree distribution histogram
plt.scatter(np.arange(1,len(x._bins)+1),np.array(x._bins),marker='x') #plot the histogram bins
plt.xlabel('Degree')
plt.ylabel('Frequency')
plt.grid()
plt.title('Degree Distribution of the Facebook Network')
plt.show()

In [ ]:
ig.plot(g,vertex_size=4,edge_width=0.5,vertex_label="",edge_color='grey') #plot the graph (will take some time)

## Community Structure Visualization

In [ ]:
ego = g.induced_subgraph(g.neighbors(484)+[484]) #Personalized network for user ID 484
print("Number of nodes:",ego.vcount()) #Print number of vertices
print("Number of edges:", ego.ecount()) #Print number of edges

In [ ]:
ig.plot(ego,vertex_size=4,edge_width=0.5,vertex_label="",edge_color='grey') #Plot the personalized network for user ID 484

In [ ]:
communities = ego.community_multilevel() #detect communities
ig.plot(communities, mark_groups = True,vertex_size=4,edge_width=0.5,vertex_label="",edge_color='grey') #plot the community structure

In [ ]:
print("Modularity for clustering of the personalized network:", communities.modularity) #find modularity of the clustering, range: 0 to 1

In [ ]:
ego = g.induced_subgraph(g.neighbors(0)+[0]) #Personalized network for user ID 0
print("Number of nodes:",ego.vcount()) #Print number of vertices
print("Number of edges:", ego.ecount()) #Print number of edges
ig.plot(ego,vertex_size=4,edge_width=0.5,vertex_label="",edge_color='grey') #Plot the personalized network for user ID 0

In [ ]:
communities = ego.community_multilevel() #detect communities
print("Modularity for clustering of the personalized network:", communities.modularity) #find modularity of the clustering, range: 0 to 1
ig.plot(communities, mark_groups = True,vertex_size=4,edge_width=0.5,vertex_label="",edge_color='grey') #plot the community structure

## Erdos-Renyi and Barabasi-Albert Network Inspection

In [ ]:
g1 = ig.Graph.Erdos_Renyi(n=1000,p=0.004,directed=False,loops=False) #create an ER network with 1000 nodes, probability of connectedness: 0.004
ig.plot(g1,vertex_size=4,edge_width=0.5,vertex_label="",edge_color='grey')

In [ ]:
g2 = ig.Graph.Barabasi(n=1000,m=1,directed=False) #create a BA network with 1000 nodes, out degree of each new node: 1
ig.plot(g2,vertex_size=4,edge_width=0.5,vertex_label="",edge_color='grey')

In [ ]:
x = g1.degree_distribution() #get the degree distribution histogram
plt.scatter(np.arange(1,len(x._bins)+1),np.array(x._bins),marker='x') #plot the histogram bins
plt.xlabel('Degree')
plt.ylabel('Frequency')
plt.grid()
plt.title('Degree Distribution of the Erdos-Renyi Network')
plt.show()

In [ ]:
x = g2.degree_distribution() #get the degree distribution histogram
plt.scatter(np.arange(1,len(x._bins)+1),np.array(x._bins),marker='x') #plot the histogram bins
plt.xlabel('Degree')
plt.ylabel('Frequency')
plt.grid()
plt.title('Degree Distribution of the Barabasi Albert Network')
plt.show()

In [ ]:
print("Is the ER graph connected?",g1.is_connected()) #Check if the entire graph is connected
print("Is the BA graph connected?",g2.is_connected()) #Check if the entire graph is connected

In [ ]:
communities = g1.community_multilevel() #detect communities
print("Modularity for clustering of the ER graph:", communities.modularity) #find modularity of the clustering, range: 0 to 1
ig.plot(communities, mark_groups = True,vertex_size=4,edge_width=0.5,vertex_label="",edge_color='grey') #plot the community structure

In [ ]:
communities = g2.community_multilevel() #detect communities
print("Modularity for clustering of the BA graph:", communities.modularity) #find modularity of the clustering, range: 0 to 1
ig.plot(communities, mark_groups = True,vertex_size=4,edge_width=0.5,vertex_label="",edge_color='grey') #plot the community structure

## Minimum Spanning Tree and GCC

In [ ]:
el = pd.read_csv('https://raw.githubusercontent.com/swapnilsayansaha/Social_Complex_Nets/main/Project4/uber_data_graph.txt',
                 sep=' ',header=None) #load the Uber dataset, which gives location IDs and time to travel between those locations.
np.savetxt(r'uber_data_graph.txt', el.values, fmt='%d') #save the file locally
el.head() #visualize the data

In [ ]:
g = ig.Graph.Read(f = 'uber_data_graph.txt', format = 'ncol', directed = False) #create a weighted undirected graph from the data
gcc = g.components().giant() #obtain the giant connected component
print("Number of nodes and edges of GCC: ",len(gcc.vs),len(gcc.es))

In [ ]:
mst = gcc.spanning_tree(weights = gcc.es["weight"]) #find the minimum spanning tree for Uber Dataset using Prim's algorithm
ig.plot(mst,vertex_size=4,edge_width=0.5,vertex_label="",edge_color='grey') 

In [ ]:
r = requests.get('https://raw.githubusercontent.com/swapnilsayansaha/Social_Complex_Nets/main/Project4/los_angeles_censustracts.json', 
                 allow_redirects=True) #download the file containing properties of each location ID (e.g., coordinates)
open('los_angeles_censustracts.json', 'wb').write(r.content) #save the file locally

In [ ]:
#save the coordinates and name of each location ID in a dictionary
location_data = {}
with open('los_angeles_censustracts.json', 'r') as f:
    cur_data = json.loads(f.readline())
    for feature in cur_data['features']:
        coordinates = np.array(feature['geometry']['coordinates'][0][0])
        location_data[feature['properties']['MOVEMENT_ID']] = {'address': feature['properties']['DISPLAY_NAME'],
                                                          'mean_coords': np.mean(coordinates.reshape(-1,2), axis=0)}

In [ ]:
list(islice(location_data.items(), 10)) #visualize few elements from dictionary

In [ ]:
# check to see if two successive nodes on the MST are geographically near each other
mst_edges = mst.es()
print("Coordinates near the two endpoints (the centroid locations) of a few edges:")
for e in mst_edges[:10]:
    x, y = mst.vs(e.tuple[0])[0]['name'], mst.vs(e.tuple[1])[0]['name']
    print(location_data[str(x)]['address'],location_data[str(x)]['mean_coords'], 
          location_data[str(y)]['address'],location_data[str(y)]['mean_coords'])

## Djikstra and TSP

In [ ]:
fig = plt.figure(figsize=(8, 8))
m = basemap.Basemap(projection='lcc', resolution='h', 
            lat_0=34.05, lon_0=-118.4,
            width=1E5, height=1E5) #Draw map of LA
m.drawcoastlines(color='gray')
m.drawcountries(color='gray')
m.drawstates(color='gray')
m.fillcontinents(color='tan')

x, y = m(-118.4912, 34.0195)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Santa\n  Monica', fontsize=7);
x, y = m(-118.2518, 34.0488)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Downtown', fontsize=7);
x, y = m(-118.4085, 33.9416)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  LAX', fontsize=7);
x, y = m(-118.1445, 34.1478)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Pasadena', fontsize=7);
x, y = m(-118.7798, 34.0259)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Malibu', fontsize=7);
x, y = m(-118.1937, 33.7701)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Long Beach', fontsize=7);
x, y = m(-118.1270, 34.0953)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Alhambra', fontsize=7);
x, y = m(-118.2201, 33.8958)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Compton', fontsize=7);
x, y = m(-118.3884, 33.8492)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Redondo Beach', fontsize=7);
x, y = m(-118.4004, 34.0736)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Beverly Hills', fontsize=7);
x, y = m(-118.3090, 34.1808)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Burbank', fontsize=7);
x, y = m(-118.3287, 34.0928)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Hollywood', fontsize=7);
x, y = m(-118.4514, 34.1899)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Van\n  Nuys', fontsize=7);
x, y = m(-118.5301, 34.2381)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Northridge', fontsize=7);
x, y = m(-118.5506, 34.1495)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Tarzana', fontsize=7);
x, y = m(-118.0328, 33.9792)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Whittier', fontsize=7);
x, y = m(-118.1670, 34.0224)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  East Los Angeles', fontsize=7);
x, y = m(-118.2551, 34.1425)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Glendale', fontsize=7);
x, y = m(-118.2251, 33.9817)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Huntington Park', fontsize=7);
x, y = m(-118.3406, 33.8358)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Torrance', fontsize=7);
x, y = m(-118.1332, 33.9401)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Downey', fontsize=7);
x, y = m(-118.3526, 33.9164)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Hawthorne', fontsize=7);
x, y = m(-118.3965, 34.0211)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Culver City', fontsize=7);
x, y = m(-118.3531, 33.9617)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Inglewood', fontsize=7);
x, y = m(-117.9981, 33.8674)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Buena\n  Park', fontsize=7);
x, y = m(-118.0817, 33.9022)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Norwalk', fontsize=7);
x, y = m(-118.1340, 33.8536)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Lakewood', fontsize=7);
x, y = m(-118.4695, 33.9850)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Venice', fontsize=7);
x, y = m(-118.3870, 33.7445)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  RP Verdes', fontsize=7);
x, y = m(-117.993938, 33.752418)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Westminster', fontsize=7);
x, y = m(-117.9242, 33.8704)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Fullerton', fontsize=7);
x, y = m(-117.9988, 33.6595)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Huntington\n  Beach', fontsize=7);
x, y = m(-118.1228, 34.0625)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Monterey Park', fontsize=7);
x, y = m(-117.9143, 33.8366)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Anaheim', fontsize=7);
x, y = m(-117.9390, 34.0686)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  West\n  Covina', fontsize=7);
x, y = m(-118.0019, 34.1443)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Monrovia', fontsize=7);
x, y = m(-117.9687, 33.9931)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Hacienda\n  Heights', fontsize=7);
x, y = m(-118.5426, 34.3917)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Santa Clarita', fontsize=7);
x, y = m(-118.5426, 34.3917)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Santa Clarita', fontsize=7);
x, y = m(-118.8376, 34.1706)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Thousand Oaks', fontsize=7);
x, y = m(-118.7815, 34.2694)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Simi Valley', fontsize=7);
x, y = m(-118.6090, 34.1654)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Woodland Hills', fontsize=7);
x, y = m(-118.4572, 34.3058)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Sylmar', fontsize=7);
x, y = m(-118.5022, 34.2794)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Granada Hills', fontsize=7);
x, y = m(-118.5361, 34.2011)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Reseda', fontsize=7);
x, y = m(-118.4424, 34.2276)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Panorama City', fontsize=7);
x, y = m(-118.3813, 34.1870)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, ' North\n Hollywood', fontsize=7);
x, y = m(-118.4595, 34.1002)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Bel Air', fontsize=7);
x, y = m(-118.1313, 34.1902)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Altadena', fontsize=7);
x, y = m(-118.6059, 34.2083)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Canoga\n  Park', fontsize=7);
x, y = m(-118.6148, 34.2506)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Chatsworth', fontsize=7);
x, y = m(-118.6615, 34.1367)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Calabasas', fontsize=7);
x, y = m(-117.9076, 34.1336)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Azusa', fontsize=7);
x, y = m(-118.4192, 33.9754)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Playa Vista', fontsize=7);
x, y = m(-118.3287, 34.0416)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Mid\n  City', fontsize=7);
x, y = m(-118.4455, 34.0635)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Westwood', fontsize=7);

In [ ]:
# Find the closest point in the Uber dataset that corresponds to Malibu and Long Beach coordinates
malibu = [34.0259, -118.7798] #actual coordinates of Malibu
long_beach =  [33.7701, -118.1937] #actual coordinates of Malibu
min_long_beach=np.inf
min_malibu=np.inf
long_beach_node=0
malibu_node=0
for i in range(1,len(location_data)+1):
    long_beach_closest=np.sqrt(((location_data[str(i)]['mean_coords'][1])-long_beach[0])**2
                               +((location_data[str(i)]['mean_coords'][0])-long_beach[1])**2)
    malibu_closest=np.sqrt((malibu[0]-location_data[str(i)]['mean_coords'][1])**2+
                           (malibu[1]-location_data[str(i)]['mean_coords'][0])**2)
    if long_beach_closest<min_long_beach:
        min_long_beach=long_beach_closest
        long_beach_node=i
    if malibu_closest<min_malibu:
        min_malibu=malibu_closest
        malibu_node=i

In [ ]:
#Calculate shortest path between Malibu and Long Beach using Dijkstra's algorihm
vid = gcc.get_all_shortest_paths(malibu_node,long_beach_node,weights=gcc.es['weight'])
vid[0].pop(0)
vid[0].pop(-1)
vid[0].sort(reverse=True)

In [ ]:
#plot the shortest path on the map:

fig = plt.figure(figsize=(8, 8))
m = basemap.Basemap(projection='lcc', resolution='h', 
            lat_0=34.05, lon_0=-118.4,
            width=1E5, height=1E5) #Draw map of LA
m.drawcoastlines(color='gray')
m.drawcountries(color='gray')
m.drawstates(color='gray')
m.fillcontinents(color='tan')

x, y = m(-118.4912, 34.0195)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Santa\n  Monica', fontsize=7);
x, y = m(-118.2518, 34.0488)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Downtown', fontsize=7);
x, y = m(-118.4085, 33.9416)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  LAX', fontsize=7);
x, y = m(-118.1445, 34.1478)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Pasadena', fontsize=7);
x, y = m(-118.7798, 34.0259)
plt.plot(x, y, 'ok', markersize=5,color='red')
plt.text(x, y, '  Malibu', fontsize=7);
x, y = m(-118.1937, 33.7701)
plt.plot(x, y, 'ok', markersize=5, color = 'red')
plt.text(x, y, '  Long Beach', fontsize=7);
x, y = m(-118.1270, 34.0953)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Alhambra', fontsize=7);
x, y = m(-118.2201, 33.8958)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Compton', fontsize=7);
x, y = m(-118.3884, 33.8492)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Redondo Beach', fontsize=7);
x, y = m(-118.4004, 34.0736)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Beverly Hills', fontsize=7);
x, y = m(-118.3090, 34.1808)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Burbank', fontsize=7);
x, y = m(-118.3287, 34.0928)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Hollywood', fontsize=7);
x, y = m(-118.4514, 34.1899)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Van\n  Nuys', fontsize=7);
x, y = m(-118.5301, 34.2381)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Northridge', fontsize=7);
x, y = m(-118.5506, 34.1495)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Tarzana', fontsize=7);
x, y = m(-118.0328, 33.9792)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Whittier', fontsize=7);
x, y = m(-118.1670, 34.0224)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  East Los Angeles', fontsize=7);
x, y = m(-118.2551, 34.1425)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Glendale', fontsize=7);
x, y = m(-118.2251, 33.9817)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Huntington Park', fontsize=7);
x, y = m(-118.3406, 33.8358)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Torrance', fontsize=7);
x, y = m(-118.1332, 33.9401)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Downey', fontsize=7);
x, y = m(-118.3526, 33.9164)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Hawthorne', fontsize=7);
x, y = m(-118.3965, 34.0211)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Culver City', fontsize=7);
x, y = m(-118.3531, 33.9617)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Inglewood', fontsize=7);
x, y = m(-117.9981, 33.8674)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Buena\n  Park', fontsize=7);
x, y = m(-118.0817, 33.9022)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Norwalk', fontsize=7);
x, y = m(-118.1340, 33.8536)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Lakewood', fontsize=7);
x, y = m(-118.4695, 33.9850)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Venice', fontsize=7);
x, y = m(-118.3870, 33.7445)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  RP Verdes', fontsize=7);
x, y = m(-117.993938, 33.752418)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Westminster', fontsize=7);
x, y = m(-117.9242, 33.8704)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Fullerton', fontsize=7);
x, y = m(-117.9988, 33.6595)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Huntington\n  Beach', fontsize=7);
x, y = m(-118.1228, 34.0625)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Monterey Park', fontsize=7);
x, y = m(-117.9143, 33.8366)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Anaheim', fontsize=7);
x, y = m(-117.9390, 34.0686)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  West\n  Covina', fontsize=7);
x, y = m(-118.0019, 34.1443)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Monrovia', fontsize=7);
x, y = m(-117.9687, 33.9931)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Hacienda\n  Heights', fontsize=7);
x, y = m(-118.5426, 34.3917)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Santa Clarita', fontsize=7);
x, y = m(-118.5426, 34.3917)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Santa Clarita', fontsize=7);
x, y = m(-118.8376, 34.1706)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Thousand Oaks', fontsize=7);
x, y = m(-118.7815, 34.2694)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Simi Valley', fontsize=7);
x, y = m(-118.6090, 34.1654)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Woodland Hills', fontsize=7);
x, y = m(-118.4572, 34.3058)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Sylmar', fontsize=7);
x, y = m(-118.5022, 34.2794)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Granada Hills', fontsize=7);
x, y = m(-118.5361, 34.2011)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Reseda', fontsize=7);
x, y = m(-118.4424, 34.2276)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Panorama City', fontsize=7);
x, y = m(-118.3813, 34.1870)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, ' North\n Hollywood', fontsize=7);
x, y = m(-118.4595, 34.1002)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Bel Air', fontsize=7);
x, y = m(-118.1313, 34.1902)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Altadena', fontsize=7);
x, y = m(-118.6059, 34.2083)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Canoga\n  Park', fontsize=7);
x, y = m(-118.6148, 34.2506)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Chatsworth', fontsize=7);
x, y = m(-118.6615, 34.1367)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Calabasas', fontsize=7);
x, y = m(-117.9076, 34.1336)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Azusa', fontsize=7);
x, y = m(-118.4192, 33.9754)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Playa Vista', fontsize=7);
x, y = m(-118.3287, 34.0416)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Mid\n  City', fontsize=7);
x, y = m(-118.4455, 34.0635)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Westwood', fontsize=7);

for i in range(len(vid[0])):
  x, y = m(location_data[str(vid[0][i])]['mean_coords'][0], location_data[str(vid[0][i])]['mean_coords'][1])
  plt.plot(x, y, 'ok', markersize=5,color='blue')
  plt.text(x, y, '  '+str(i), fontsize=10); 

In [ ]:
#Let's create the graph and extract MST using networkX
arr = el
arr1 = arr.rename(columns={0: "source", 1: "sink", 2: "weight"})
g = nx.from_pandas_edgelist(arr1, 'source','sink', ['weight'])
gcc = g.subgraph(max(nx.connected_components(g), key=len))
mst = nx.minimum_spanning_tree(gcc)
mg = nx.MultiGraph()
mst_cost = 0
for i in mst.edges:
    w = mst.edges[i[0],i[1]]['weight']
    mst_cost += w
    mg.add_edge(i[0],i[1],weight=w)
    mg.add_edge(i[0],i[1],weight=w)

vertices, count = [], 0
for i in mg.nodes:
    vertices.append(i)
    count += 1
    if count>60: 
        break

In [ ]:
# TSP using 2-approximate algorithm (will take a long time to run)

costs, cur_paths = [], []
for vertex in vertices:
    tour = [u for u,v in nx.eulerian_circuit(mg,source=vertex)]
    cur_path, visited_nodes = [], set()
    for i in tour:
        if i not in visited_nodes: 
            cur_path.append(i) 
            visited_nodes.add(i) 
    cur_path.append(cur_path[0])
    cur_paths.append(cur_path)

    approx_cost = 0
    for i in range(len(cur_path)-1):
        s,t = cur_path[i], cur_path[i+1]
        w = 0
        if mst.has_edge(s,t): 
            w = mst.edges[s,t]['weight']
        else: 
            w = nx.dijkstra_path_length(gcc,s,t)
        approx_cost += w
    costs.append(approx_cost)

min_approx_cost = min(costs)
trajectory = cur_paths[np.argmin(costs)]

In [ ]:
# Extract coordinates in order from TSP output

data = json.load(open('los_angeles_censustracts.json'))
location_data = []
for i in trajectory:
    for j in range(len(data['features'])):
        if data['features'][j]['properties']['MOVEMENT_ID']==str(int(i)):
            cur_loc = data['features'][j]['geometry']['coordinates'][0]
            if len(cur_loc)==1:
                t = np.asarray(cur_loc[0]).mean(axis=0)
                location_data.append(t)
            elif len(cur_loc)==2:
                t = np.asarray(cur_loc[0]+cur_loc[1]).mean(axis=0)
                location_data.append(t)
            elif i==1932.0:
                t = np.asarray(cur_loc[0]+cur_loc[1]+cur_loc[2]+cur_loc[3]+cur_loc[4]+cur_loc[5]).mean(axis=0)
                location_data.append(t)
            else: 
                t = np.asarray(cur_loc).mean(axis=0)
                location_data.append(t)
x,y = [i[0] for i in location_data], [i[1] for i in location_data]

In [ ]:
#plot on the map of Los Angeles
lon = x
lat = y

fig, ax = plt.subplots(figsize = (8,8))
m = basemap.Basemap(projection='lcc', resolution='h', lat_0=34.05, lon_0=-118.25,width=1E5, height=1E5) #Draw map of LA
m.drawcoastlines(color='gray')
m.drawcountries(color='gray')
m.drawstates(color='gray')
m.fillcontinents(color='tan')
x, y = m(-118.4912, 34.0195)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Santa\n  Monica', fontsize=7);
x, y = m(-118.2518, 34.0488)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Downtown', fontsize=7);
x, y = m(-118.4085, 33.9416)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  LAX', fontsize=7);
x, y = m(-118.1445, 34.1478)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Pasadena', fontsize=7);
x, y = m(-118.7798, 34.0259)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Malibu', fontsize=7);
x, y = m(-118.1937, 33.7701)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Long Beach', fontsize=7);
x, y = m(-118.1270, 34.0953)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Alhambra', fontsize=7);
x, y = m(-118.2201, 33.8958)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Compton', fontsize=7);
x, y = m(-118.3884, 33.8492)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Redondo Beach', fontsize=7);
x, y = m(-118.4004, 34.0736)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Beverly Hills', fontsize=7);
x, y = m(-118.3090, 34.1808)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Burbank', fontsize=7);
x, y = m(-118.3287, 34.0928)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Hollywood', fontsize=7);
x, y = m(-118.4514, 34.1899)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Van\n  Nuys', fontsize=7);
x, y = m(-118.5301, 34.2381)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Northridge', fontsize=7);
x, y = m(-118.5506, 34.1495)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Tarzana', fontsize=7);
x, y = m(-118.0328, 33.9792)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Whittier', fontsize=7);
x, y = m(-118.1670, 34.0224)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  East Los Angeles', fontsize=7);
x, y = m(-118.2551, 34.1425)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Glendale', fontsize=7);
x, y = m(-118.2251, 33.9817)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Huntington Park', fontsize=7);
x, y = m(-118.3406, 33.8358)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Torrance', fontsize=7);
x, y = m(-118.1332, 33.9401)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Downey', fontsize=7);
x, y = m(-118.3526, 33.9164)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Hawthorne', fontsize=7);
x, y = m(-118.3965, 34.0211)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Culver City', fontsize=7);
x, y = m(-118.3531, 33.9617)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Inglewood', fontsize=7);
x, y = m(-117.9981, 33.8674)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Buena\n  Park', fontsize=7);
x, y = m(-118.0817, 33.9022)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Norwalk', fontsize=7);
x, y = m(-118.1340, 33.8536)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Lakewood', fontsize=7);
x, y = m(-118.4695, 33.9850)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Venice', fontsize=7);
x, y = m(-118.3870, 33.7445)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  RP Verdes', fontsize=7);
x, y = m(-117.993938, 33.752418)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Westminster', fontsize=7);
x, y = m(-117.9242, 33.8704)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Fullerton', fontsize=7);
x, y = m(-117.9988, 33.6595)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Huntington\n  Beach', fontsize=7);
x, y = m(-118.1228, 34.0625)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Monterey Park', fontsize=7);
x, y = m(-117.9143, 33.8366)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Anaheim', fontsize=7);
x, y = m(-117.9390, 34.0686)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  West\n  Covina', fontsize=7);
x, y = m(-118.0019, 34.1443)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Monrovia', fontsize=7);
x, y = m(-117.9687, 33.9931)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Hacienda\n  Heights', fontsize=7);
x, y = m(-118.5426, 34.3917)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Santa Clarita', fontsize=7);
x, y = m(-118.5426, 34.3917)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Santa Clarita', fontsize=7);
x, y = m(-118.8376, 34.1706)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Thousand Oaks', fontsize=7);
x, y = m(-118.7815, 34.2694)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Simi Valley', fontsize=7);
x, y = m(-118.6090, 34.1654)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Woodland Hills', fontsize=7);
x, y = m(-118.4572, 34.3058)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Sylmar', fontsize=7);
x, y = m(-118.5022, 34.2794)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Granada Hills', fontsize=7);
x, y = m(-118.5361, 34.2011)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Reseda', fontsize=7);
x, y = m(-118.4424, 34.2276)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Panorama City', fontsize=7);
x, y = m(-118.3813, 34.1870)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, ' North\n Hollywood', fontsize=7);
x, y = m(-118.4595, 34.1002)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Bel Air', fontsize=7);
x, y = m(-118.1313, 34.1902)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Altadena', fontsize=7);
x, y = m(-118.6059, 34.2083)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Canoga\n  Park', fontsize=7);
x, y = m(-118.6148, 34.2506)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Chatsworth', fontsize=7);
x, y = m(-118.6615, 34.1367)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Calabasas', fontsize=7);
x, y = m(-117.9076, 34.1336)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Azusa', fontsize=7);
x, y = m(-118.4192, 33.9754)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Playa Vista', fontsize=7);
x, y = m(-118.3287, 34.0416)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Mid\n  City', fontsize=7);
x, y = m(-118.4455, 34.0635)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, '  Westwood', fontsize=7);


x,y = m(lon, lat)
plt.plot(x,y,'o-',color='blue',markersize=2,markerfacecolor='red')
plt.show()